In [1]:
import pandas as pd
import geopandas as gpd
from shapely.ops import nearest_points

### Load data

In [10]:
hosp_df = pd.read_csv('../data/yearly_hospital_lists/geocoded/final/combined_with_coords_final.csv')

hosp_gdf = gpd.GeoDataFrame(
    hosp_df,
    geometry=gpd.points_from_xy(hosp_df['final_longitude'], hosp_df['final_latitude']),
    crs="EPSG:4326"  # lat/lon CRS
)

# Reproject to NAD83 / Conus Albers for distance (meters)
hosp_gdf = hosp_gdf.to_crs("EPSG:5070")

/var/folders/8c/gxrlw_4x0w7d7z884599dvbr0000gn/T/ipykernel_98905/1706764617.py:1: DtypeWarning: Columns (0,5,6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  hosp_df = pd.read_csv('../data/yearly_hospital_lists/geocoded/final/combined_with_coords_final.csv')


In [11]:
# Load and reproject county centroids
county_gdf = gpd.read_file('../data/county_pop_centroids/nhgis0001_shape/nhgis0001_shapefile_cenpop2020_us_county_cenpop_2020/US_county_cenpop_2020.shp')

# Reproject to NAD83 / Conus Albers for distance (meters)
proj_crs = "EPSG:5070"  # NAD83 / Conus Albers (unit = meters)
county_gdf = county_gdf.to_crs(proj_crs)

In [12]:
# Urban/rural classifications
urban_rural_df = pd.read_csv('../data/urban_rural/data-table.csv', dtype={'FIPS code': str})
urban_rural_df['FIPS_clean'] = urban_rural_df['FIPS code'].str.zfill(5)

### Join urban-rural classifications to counties

In [13]:
# Make FIPS code for counties
county_gdf['FIPS'] = county_gdf['STATEFP'] + county_gdf['COUNTYFP']

# Merge
county_gdf_merged = county_gdf.merge(urban_rural_df, how='left', left_on='FIPS', right_on='FIPS_clean')

In [14]:
# See states where the join didn't work
county_gdf_merged[county_gdf_merged['2023 Code'].isna()]['STNAME'].unique()

array(['Connecticut', 'Puerto Rico'], dtype=object)

### Distance analysis

In [15]:
# Define function
def compute_nearest_distance(hosp_gdf_subset, county_gdf):
    if hosp_gdf_subset.empty:
        return pd.Series([None] * len(county_gdf), index=county_gdf.index)

    hospital_union = hosp_gdf_subset.geometry.union_all()
    distances = county_gdf.geometry.apply(
        lambda geom: geom.distance(nearest_points(geom, hospital_union)[1])
    )
    return distances / 1609.34  # meters to miles

In [16]:
# Loop over years and compute distances

output = pd.DataFrame()
output["FIPS"] = county_gdf["FIPS"]

for year in range(2016, 2025):
    # Filter the hospital GeoDataFrame to just acute hospitals for that year
    hosp_year = hosp_gdf[(hosp_gdf["YEAR"] == year) & (hosp_gdf["HOSPITAL_TYPE"] == "Acute")]

    # Compute distances from each county to its nearest acute hospital
    distances = compute_nearest_distance(hosp_year, county_gdf)
    
    # Store the resulting series as a new col in output table
    output[f"distance_miles_{year}"] = distances
    print(f"Finished {year}")

Finished 2016
Finished 2017
Finished 2018
Finished 2019
Finished 2020
Finished 2021
Finished 2022
Finished 2023
Finished 2024


In [22]:
len(output['FIPS'])

3221

In [23]:
county_gdf

,GISJOIN,GEOID,STATEFP,COUNTYFP,STNAME,COUNAME,POPULATION,LATITUDE,LONGITUDE,geometry,FIPS
0,G0100010,01001,01,001,Alabama,Autauga,58805,32.500194,-86.487813,POINT (887485.091 1091989.207),01001
1,G0100030,01003,01,003,Alabama,Baldwin,231767,30.537396,-87.761478,POINT (787890.83 863218.563),01003
2,G0100050,01005,01,005,Alabama,Barbour,25223,31.843981,-85.301306,POINT (1005960.112 1031085.25),01005
3,G0100070,01007,01,007,Alabama,Bibb,22293,33.032236,-87.136826,POINT (821586.755 1145231.695),01007
4,G0100090,01009,01,009,Alabama,Blount,59134,33.954604,-86.592667,POINT (861687.94 1252587.337),01009
...,...,...,...,...,...,...,...,...,...,...,...
3216,G7201450,72145,72,145,Puerto Rico,Vega Baja,54414,18.439692,-66.399082,POINT (3193399.001 10312.777),72145
3217,G7201470,72147,72,147,Puerto Rico,Vieques,8249,18.134221,-65.448953,POINT (3302688.508 11849.919),72147
3218,G7201490,72149,72,149,Puerto Rico,Villalba,22093,18.123806,-66.486557,POINT (3194554.685 -24674.139),72149
3219,G7201510,72151,72,151,Puerto Rico,Yabucoa,30426,18.068957,-65.885852,POINT (3259226.983 -9831.819),72151
